In [1]:
import pandas as pd
from pathlib import Path

In [3]:
model = 'llama3-8b-IS'
dist_folder = 'no-distil-fine-tune'
filename = 'FINAL_Completed(0 - 1319).csv'

In [4]:
basepath = (Path() / '..' /'..'/ 'logs' / model / dist_folder).resolve()

In [5]:
df = pd.read_csv(basepath / filename)

In [6]:
ex_df = pd.read_csv((Path() / '..' / '..' / 'logs' / 'gemma2b-it' / 'no-distil-test' / 'FINAL_RESULTS.csv').resolve())

In [7]:
df['extracted_answer'] = ex_df['extracted_answer']

In [8]:
shot8prompt = "Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there\nwill be 21 trees. How many trees did the grove workers plant today?\nA: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted. So,\nthey must have planted 21 - 15 = 6 trees. The answer is 6.\n### 6\nQ: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nA: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.\n### 5\nQ: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\nA: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74 chocolates.\n### 74\n35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.\nQ: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did\nJason give to Denny?\nA: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of\nlollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.\n### 8\nQ: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he\nhave now?\nA: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so in\ntotal he has 7 + 2 = 9 toys. The answer is 9.\n### 9\nQ: There were nine computers in the server room. Five more computers were installed each day, from monday\nto thursday. How many computers are now in the server room?\nA: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 = 20\ncomputers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers. The\nanswer is 29.\n### 29\nQ: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf\nballs did he have at the end of wednesday?\nA: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday\nhe lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.\n### 33\nQ: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\nA: She bought 5 bagels for $3 each. This means she spent 5 * $3 = $15 on the bagels. She had $23 in beginning,\nso now she has $23 - $15 = $8. The answer is 8\n### 8\nQ: "

In [9]:
for i, row in df.iterrows():
    df.loc[i, 'model_answer'] = row['model_answer'].removeprefix(shot8prompt)
    df.loc[i, 'model_answer'] = df['model_answer'][i].removeprefix(row['question'])
    df.loc[i, 'model_answer'] = df['model_answer'][i].removeprefix('\n Please respond with just the answer. The answer is:')
    df.loc[i, 'model_answer'] = df['model_answer'][i].strip()

    # df.loc[i, 'extracted_answer'] = row['answer'].split('####')[1].strip()

In [10]:
empt = 0
for i, row in df.iterrows():
    if not row['model_answer']:
        empt += 1
empt

0

In [11]:
import re
df['model_ex1'] = df['model_answer']
for i, row in df.iterrows():
    sepstr_extr = re.findall(r'oxed{ *([,\d]+\.?[,\d]*) *}', row['model_answer'])
    first_line = row['model_answer'].split('\n')[0]
    mtches = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n|.){1}', first_line)
    osepstr = re.findall(r'(the ?(final|correct)* ?answer ?is)', row['model_answer'])
    oind = row['model_answer'].find(osepstr[0][0]) if osepstr else -1
    oextr = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n|.){1}', row['model_answer'][oind:]) if oind != -1 else []
    if sepstr_extr:
        v = re.sub(r',', '', sepstr_extr[0])
    
    elif len(mtches) == 1 or len(set([v[0] for v in mtches])) == 1:
        v = re.sub(r',', '', mtches[0][0])
    elif oextr:
        v = re.sub(r',', '', extr[0][0])
    else:
        extr = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n|.){1}', row['model_answer'])
        if len(extr) == 0:
            df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])
            continue
        v = re.sub(r',', '', extr[-1][0])
    try:
        df.loc[i, 'model_ex1'] = float(v)
    except:
        df.loc[i, 'model_ex1'] = df['model_ex1'][i]
    df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])

In [37]:
import re

df['model_ex1'] = df['model_answer']
for i, row in df.iterrows():
    extr = re.finditer(r'([^+\-\/*]|^)[$]?([-+]?[,\d]+\.?[,\d]*)\%?($|\n|\.){1}', row['model_answer'])
    estrs = [(m.group(2), m.start()) for m in extr]
    print(estrs) if i ==0 else ''
    if len(estrs) > 1:  

        if estrs[0][1] == 0:
            v = re.sub(r',', '', estrs[0][0])
            
        else:
            df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])
            continue
    elif len(estrs) == 0:
        df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])
        continue
    else:
        v = re.sub(r',', '', estrs[0][0])
    try:
        df.loc[i, 'model_ex1'] = float(v)
    except:
        df.loc[i, 'model_ex1'] = v
    df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])


[('04', 0)]


In [12]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [16]:
df['model_ex1_t'].value_counts()

model_ex1_t
<class 'float'>    1185
<class 'str'>       134
Name: count, dtype: int64

In [17]:
def save_df(df : pd.DataFrame, model : str, dist_folder : str, edition : str):
    base_path = Path()
    base_path = base_path / '..' /'..'/ 'logs' / model / dist_folder / f'{edition}.csv'
    df.to_csv(base_path)

In [18]:
for i, row in df[df['model_ex1_t'] != float].iterrows():
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}. \n \n Question: {row['question']}')
    
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        if inp == 'no':
            break
        print(f'{i} could not be parsed')

In [19]:
for i, row in df.iterrows():
    df.loc[i, 'model_ex1_t'] = type(row['model_ex1'])


In [21]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [22]:
df['correct'].value_counts()

correct
False    1016
True      303
Name: count, dtype: int64

In [108]:
import ipywidgets as widgets
from random import randint

@widgets.interact()
def resp_output():
    print(randint(1,6))

@widgets.interact(val='')
def get_input(val : str):
    print(val)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

interactive(children=(Text(value='', description='val'), Output()), _dom_classes=('widget-interact',))

In [104]:
import ipywidgets as widgets
info = [
widgets.Text(
    value=str(0),
    description='I:',
    disabled=True   
), 
widgets.Output(
    
)

]
items = [
widgets.Text(
    value='',
    description='Value:',
    disabled=False   
), 
widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check-circle' # (FontAwesome names without the `fa-` prefix)
)
]

widgets.HBox(items)

In [48]:
for i, row in df[df['model_ex1_t'] != float].iterrows():
    print(f'Model Answer: {row['model_ex1']}')
    print(f'True Answer: {row['answer']}')
    print(f'Question: {row['question']}')
    print()
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}')
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        print(f'{i} could not be parsed')
    df.loc[i, 'model_ex1_t'] = type(row['model_ex1'])
    

Model Answer: 2.5 bolts of blue fiber and 1.25 bolts of white fiber.

True Answer: It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3
Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?

Model Answer: 
True Answer: If the total calorie target is 2000 and I have consumed 1800 calories then I have 2000-1800 = <<2000-1800=200>>200 calories left to eat
If each serving of chips has 250 calories and I only have 200 calories left to eat, then I can only eat 200/250 of a serving = 4/5 of a serving
We also know that a 300g bag of chips has 5 servings, hence each serving has 300g/5 = <<300/5=60>>60 grams
If I can only eat 4/5 of a serving, then I can eat only 60g * 4/5 = 240g/5 = 48 grams
#### 48
Question: According to its nutritional info, a bag of chips has 250 calories per serving. If a 300g bag has 5 servings, how many grams can you eat if your daily calorie target 

In [164]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [165]:
df['correct'].value_counts()

correct
False    1157
True      162
Name: count, dtype: int64

In [133]:
prev_dr['correct'].value_counts()

correct
False    1191
True      128
Name: count, dtype: int64

In [23]:
save_df(df, model, dist_folder, 'FINAL_RESULTS_LAST_ANS')

In [310]:
for i, row in df.iterrows():
    if row['extracted_answer'] < 0:
        print(i)

489
1113


In [95]:
df['model_ex1_TRUE'] = ''
extrs = df[df['model_ex1_t'] == float].sample(50)

for i, row in extrs.iterrows():
    v = input(f'Extraced Answer: {row['model_ex1']} | Model Answer: {row['model_answer']} | Question: {row['question']}')
    if v:
        df.loc[i, 'model_ex1_TRUE'] = False
    else:
        df.loc[i, 'model_ex1_TRUE'] = True
print(extrs.index)

Index([ 555,  677,  526,  117,  790, 1234,  812,  741,  542, 1144,  213,  268,
         24,  222,  324,  423, 1277,  381,  147,  181,  592,   27,  972,  403,
        524,  451,  164, 1222,  847,   54,  549,  122,  601,  477, 1192,  766,
        138,  380,  669,  998, 1275,  355, 1313,  861,  782,  800,  908,  866,
        833, 1263],
      dtype='int64')


[ 743,  851,  658, 1273,  587,   15,  480,  539, 1087,  265,   92,  182,
         35,   12,  151, 1070, 1244,  354, 1067, 1073,  617,  373,  763, 1174,
        949, 1308, 1156,  445,  555, 1291,  446, 1098,   22,  156,  678,  405,
       1184,   29,  305,  667,  523,  190,  296,  142,  146,  285,   37, 1161,
        881,  141]



In [83]:
df.loc[1152][['model_answer', 'model_ex1']]

model_answer    0.08 (8%). The explanation is below.
model_ex1       0.08 (8%). The explanation is below.
Name: 1152, dtype: object

In [67]:
df[df['model_ex1_TRUE'] == False][['model_answer', 'model_ex1']].head()

,model_answer,model_ex1
109,$30.00. Thank you.,0.0
110,2.4. Thank you.,4.0
358,$5.00.,0.0
758,$132.00.,0.0
1152,0.08 (8%). The explanation is below.,0.08
